In [2]:
import os
import re
import numpy as np
from scipy import ndimage

import sys
import pickle
import json
import tables

import importlib.util #import from superordinate folder
spec =importlib.util.spec_from_file_location("helper",str(re.search("(.*/)",os.getcwd()).groups(0)[0])+"helper.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)

# Decode motion using decode_motion

In [3]:
videos = ["RPS_C_13_C3.wmv", "RPS_FP_13_C3.wmv", "RPS_PS_13_C3.wmv"]
identifiers = [re.search("(.*)\.wmv",x).groups()[0] for x in videos]# cut off ending
outfiles = [x+".hdf5" for x in identifiers]

In [ ]:
for vid, outfile in zip(videos,outfiles):
    ! ./decode_motion {vid} {outfile+"new"}

Input #0, asf, from 'RPS_C_13_C3.wmv':
  Metadata:
    WMFSDKNeeded    : 0.0.0.0000
    WMFSDKVersion   : 12.0.16299.15
    IsVBR           : 0
  Duration: 00:11:59.36, start: 0.000000, bitrate: 1774 kb/s
    Stream #0:0(eng): Video: h264 (Main) (H264 / 0x34363248), yuvj420p(pc, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], 50000 kb/s, 25 fps, 25 tbr, 1k tbn, 2k tbc
    Stream #0:1(eng): Audio: aac (LC) ([255][0][0][0] / 0x00FF), 16000 Hz, mono, fltp, 64 kb/s

**************************************************************************************
*       Tool : Dense Flow Decoder                                                    *
*     Author : Michael Gerstenberger (see copyright for version history and authors) *
*  Used Libs : FFmpeg, HDF5                                                          *
*Description : A tool to extract motion vectors from H264 videos and save them as    *
             : a tensor of size n_frames x width x height x 2 (for dx and dy) in     *
        

# Compute and save 1d movement signal for all ROIs for both participants

In [3]:
directory = os.getcwd() + "/"
hdf5er = np.array([x for x in os.listdir(directory)])[np.array([".hdf5" in x for x in os.listdir(directory)])]
outfolder = directory

In [4]:
#For all hdf5, for all head hands and body jsons compute weighted histograms if input was present
one_d = {}
two_d = {}
mot = {}
for file in hdf5er:
    root = re.search("(.*)\.",file).groups()[0]
    identifier = [root+"_left_head",root+"_left_hands",root+"_left_body",
                 root+"_right_head",root+"_right_hands",root+"_right_body"]
    jsons = [root+"_left_head.json",root+"_left_hands.json",root+"_left_body.json",
            root+"_right_head.json",root+"_right_hands.json",root+"_right_body.json"]
    for i,j in zip(identifier,jsons):
        try:
            with open(directory+j, encoding='utf-8') as fh:
                roi = json.load(fh)
                h264 = tables.open_file(directory+file, mode='r')
                h264 = h264.root.motion_tensor
                mothistmap = helper.weighted_histograms(h264,roi=roi)
                mot[i] = mothistmap#append mothistmap
                
                hist = ndimage.gaussian_filter(mothistmap,10)
                amount = np.sum(hist,axis=0)#1d signal
                one_d[i] = amount
                
                #Avoid multikollinearity that arises from smoothing
                left = np.sum(mothistmap[0:5,:],axis=0)+np.sum(mothistmap[31:,:],axis=0)
                down = np.sum(mothistmap[5:14,:],axis=0)
                right = np.sum(mothistmap[14:23,:],axis=0)
                up = np.sum(mothistmap[23:31,:],axis=0)
                two_d[i]= np.array([up,left,down,right])
        except Exception as e:
            print(e)

0.0
0.0055617352614015575
0.011123470522803115
0.01668520578420467
0.02224694104560623
0.027808676307007785
0.03337041156840934
0.0389321468298109
0.04449388209121246
0.05005561735261402
0.05561735261401557
0.06117908787541713
0.06674082313681869
0.07230255839822025
0.0778642936596218
0.08342602892102335
0.08898776418242492
0.09454949944382647
0.10011123470522804
0.10567296996662959
0.11123470522803114
0.1167964404894327
0.12235817575083426
0.12791991101223582
0.13348164627363737
0.13904338153503892
0.1446051167964405
0.15016685205784205
0.1557285873192436
0.16129032258064516
0.1668520578420467
0.1724137931034483
0.17797552836484984
0.1835372636262514
0.18909899888765294
0.1946607341490545
0.20022246941045607
0.20578420467185762
0.21134593993325917
0.21690767519466073
0.22246941045606228
0.22803114571746386
0.2335928809788654
0.23915461624026696
0.2447163515016685
0.25027808676307006
0.25583982202447164
0.26140155728587317
0.26696329254727474
0.2725250278086763
0.27808676307007785
0.28

0.41156840934371525
0.41713014460511677
0.42269187986651835
0.42825361512791993
0.43381535038932145
0.43937708565072303
0.44493882091212456
0.45050055617352613
0.4560622914349277
0.46162402669632924
0.4671857619577308
0.4727474972191324
0.4783092324805339
0.4838709677419355
0.489432703003337
0.4949944382647386
0.5005561735261401
0.5061179087875417
0.5116796440489433
0.5172413793103449
0.5228031145717463
0.5283648498331479
0.5339265850945495
0.5394883203559511
0.5450500556173526
0.5506117908787542
0.5561735261401557
0.5617352614015573
0.5672969966629589
0.5728587319243604
0.578420467185762
0.5839822024471635
0.5895439377085651
0.5951056729699666
0.6006674082313682
0.6062291434927698
0.6117908787541713
0.6173526140155728
0.6229143492769744
0.628476084538376
0.6340378197997776
0.639599555061179
0.6451612903225806
0.6507230255839822
0.6562847608453838
0.6618464961067854
0.6674082313681868
0.6729699666295884
0.67853170189099
0.6840934371523916
0.6896551724137931
0.6952169076751946
0.7007786

0.7953281423804227
0.8008898776418243
0.8064516129032258
0.8120133481646273
0.8175750834260289
0.8231368186874305
0.8286985539488321
0.8342602892102335
0.8398220244716351
0.8453837597330367
0.8509454949944383
0.8565072302558399
0.8620689655172413
0.8676307007786429
0.8731924360400445
0.8787541713014461
0.8843159065628476
0.8898776418242491
0.8954393770856507
0.9010011123470523
0.9065628476084538
0.9121245828698554
0.917686318131257
0.9232480533926585
0.92880978865406
0.9343715239154616
0.9399332591768632
0.9454949944382648
0.9510567296996663
0.9566184649610678
0.9621802002224694
0.967741935483871
0.9733036707452726
0.978865406006674
0.9844271412680756
0.9899888765294772
0.9955506117908788
0.0
0.0055617352614015575
0.011123470522803115
0.01668520578420467
0.02224694104560623
0.027808676307007785
0.03337041156840934
0.0389321468298109
0.04449388209121246
0.05005561735261402
0.05561735261401557
0.06117908787541713
0.06674082313681869
0.07230255839822025
0.0778642936596218
0.08342602892102

0.4732660994985633
0.478900219730689
0.4845343399628148
0.49016846019494054
0.4958025804270663
0.5014367006591921
0.5070708208913178
0.5127049411234436
0.5183390613555693
0.5239731815876951
0.5296073018198209
0.5352414220519466
0.5408755422840723
0.5465096625161981
0.5521437827483239
0.5577779029804496
0.5634120232125753
0.5690461434447011
0.5746802636768269
0.5803143839089526
0.5859485041410784
0.5915826243732041
0.5972167446053299
0.6028508648374556
0.6084849850695814
0.6141191053017071
0.6197532255338329
0.6253873457659587
0.6310214659980844
0.6366555862302101
0.6422897064623359
0.6479238266944617
0.6535579469265874
0.6591920671587131
0.6648261873908389
0.6704603076229647
0.6760944278550904
0.6817285480872162
0.6873626683193419
0.6929967885514677
0.6986309087835935
0.7042650290157192
0.7098991492478449
0.7155332694799708
0.7211673897120965
0.7268015099442222
0.7324356301763479
0.7380697504084737
0.7437038706405995
0.7493379908727252
0.754972111104851
0.7606062313369767
0.76624035156

0.8958251169079948
0.9014592371401206
0.9070933573722463
0.9127274776043721
0.9183615978364978
0.9239957180686236
0.9296298383007493
0.9352639585328751
0.9408980787650009
0.9465321989971266
0.9521663192292523
0.957800439461378
0.9634345596935039
0.9690686799256296
0.9747028001577553
0.9803369203898811
0.9859710406220069
0.9916051608541326
0.9972392810862584
[Errno 2] No such file or directory: '/mnt/c/Users/eler/Documents/Cognitive_Science/Bachelorarbeit/Scripts/processing_pipeline/dyad_13/RPS_FP_13_C3_right_head.json'
[Errno 2] No such file or directory: '/mnt/c/Users/eler/Documents/Cognitive_Science/Bachelorarbeit/Scripts/processing_pipeline/dyad_13/RPS_FP_13_C3_right_hands.json'
[Errno 2] No such file or directory: '/mnt/c/Users/eler/Documents/Cognitive_Science/Bachelorarbeit/Scripts/processing_pipeline/dyad_13/RPS_FP_13_C3_right_body.json'
HDF5 error back trace

  File "H5F.c", line 511, in H5Fopen
    unable to open file
  File "H5Fint.c", line 1604, in H5F_open
    unable to read

0.04236988913212344
0.045900713226467056
0.049431537320810674
0.0529623614151543
0.05649318550949792
0.060024009603841535
0.06355483369818515
0.06708565779252877
0.07061648188687239
0.07414730598121602
0.07767813007555964
0.08120895416990326
0.08473977826424688
0.0882706023585905
0.09180142645293411
0.09533225054727773
0.09886307464162135
0.10239389873596498
0.1059247228303086
0.10945554692465222
0.11298637101899583
0.11651719511333945
0.12004801920768307
0.12357884330202669
0.1271096673963703
0.13064049149071394
0.13417131558505754
0.13770213967940118
0.14123296377374478
0.1447637878680884
0.14829461196243204
0.15182543605677565
0.15535626015111928
0.15888708424546288
0.16241790833980652
0.16594873243415012
0.16947955652849375
0.17301038062283736
0.176541204717181
0.18007202881152462
0.18360285290586822
0.18713367700021186
0.19066450109455546
0.1941953251888991
0.1977261492832427
0.20125697337758633
0.20478779747192996
0.20831862156627357
0.2118494456606172
0.2153802697549608
0.218911

0.5402160864345739
0.5437469105289174
0.547277734623261
0.5508085587176047
0.5543393828119483
0.557870206906292
0.5614010310006355
0.5649318550949791
0.5684626791893228
0.5719935032836664
0.5755243273780101
0.5790551514723536
0.5825859755666972
0.5861167996610409
0.5896476237553845
0.5931784478497282
0.5967092719440718
0.6002400960384153
0.603770920132759
0.6073017442271026
0.6108325683214463
0.6143633924157899
0.6178942165101334
0.6214250406044771
0.6249558646988207
0.6284866887931643
0.632017512887508
0.6355483369818515
0.6390791610761952
0.6426099851705388
0.6461408092648824
0.6496716333592261
0.6532024574535696
0.6567332815479133
0.6602641056422569
0.6637949297366005
0.6673257538309442
0.6708565779252877
0.6743874020196314
0.677918226113975
0.6814490502083186
0.6849798743026623
0.6885106983970058
0.6920415224913494
0.6955723465856931
0.6991031706800367
0.7026339947743804
0.706164818868724
0.7096956429630675
0.7132264670574112
0.7167572911517548
0.7202881152460985
0.7238189393404421

In [8]:
if(False):#check for multicolineraity as a result of smoothing 4xn representation
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sns

    for roi in ["hands","head","body"]:
        fig, ax = plt.subplots(1)
        ax.set_title(roi)
        df = pd.DataFrame()
        up = two_d["RPS_C_13_C3_left_"+roi][0,2000:3000]
        left = two_d["RPS_C_13_C3_left_"+roi][1,2000:3000]
        down = two_d["RPS_C_13_C3_left_"+roi][2,2000:3000]
        right = two_d["RPS_C_13_C3_left_"+roi][3,2000:3000]

        df["up"]=up
        df["left"]=left
        df["down"]=down
        df["right"]=right

        sns.heatmap(data=df.corr().round(2), annot=True)
    print("Conclusion: There is a high intercorrelation for directions especially for body and for hands")

In [9]:
with open(outfolder+"1d_movement.motion","wb") as f:
    pickle.dump(one_d,f)#1d signal

In [10]:
with open(outfolder+"2d_movement.motion","wb") as f:
    pickle.dump(two_d,f)#4xN signal for up left down right

In [11]:
# Test loading
with open(outfolder+"1d_movement.motion","rb") as f:
    one_d = pickle.load(f)

In [12]:
print(one_d.keys())

dict_keys(['RPS_C_13_C3_left_head', 'RPS_C_13_C3_left_hands', 'RPS_C_13_C3_left_body', 'RPS_C_13_C3_right_head', 'RPS_C_13_C3_right_hands', 'RPS_C_13_C3_right_body', 'RPS_FP_13_C3_left_head', 'RPS_FP_13_C3_left_hands', 'RPS_FP_13_C3_left_body', 'RPS_PS_13_C3_left_head', 'RPS_PS_13_C3_left_hands', 'RPS_PS_13_C3_left_body'])


In [27]:
np.corrcoef(two_d["RPS_C_13_C3_left_head"][0], one_d["RPS_C_13_C3_left_head"])

array([[1.        , 0.56429559],
       [0.56429559, 1.        ]])

# Compute band power to common average 
### (Rereferencing & alignment to video start trigger)

In [34]:
folder = "/mnt/c/Users/eler/Documents/Cognitive_Science/Bachelorarbeit/Scripts/processing_pipeline/dyad_13/"

In [40]:
"""electrodes = ['V1_1', 'V2_1', 'F7_1', 'F3_1', 'Fz_1', 'F4_1', 'F8_1', 'FC5_1', 'FC1_1', 'FCz_1',
       'FC2_1', 'FC6_1', 'T7_1', 'C3_1', 'Cz_1', 'C4_1', 'T8_1', 'TP9_1', 'CP5_1', 'CP1_1',
       'CP2_1', 'CP6_1', 'TP10_1', 'P7_1', 'P3_1', 'Pz_1', 'P4_1', 'P8_1', 'H1_1', 'O1_1',
       'Oz_1', 'O2_1', 'H2_1']
electrodes.extend(['V1_2', 'V2_2', 'F7_2', 'F3_2', 'Fz_2', 'F4_2', 'F8_2', 'FC5_2', 'FC1_2', 'FCz_2',
       'FC2_2', 'FC6_2', 'T7_2', 'C3_2', 'Cz_2', 'C4_2', 'T8_2', 'TP9_2', 'CP5_2', 'CP1_2',
       'CP2_2', 'CP6_2', 'TP10_2', 'P7_2', 'P3_2', 'Pz_2', 'P4_2', 'P8_2', 'H1_2', 'O1_2',
       'Oz_2', 'O2_2', 'H2_2'])"""

electrodes_from_cap = ["V1","V2","F7","F3","Fz","F4","F8","FC5","FC1",
                       "FC2","FC6","T7","C3","Cz","C4","T8","FCz","CP5",
                       "CP1","CP2","CP6","TP10","P7","P3","Pz","P4","P8","H1","O1","Oz","O2","H2"]
exclude = [0,1,27,31,3,6,17,20]

In [36]:
"""electrodes = ['FZ_1', 'F3_1', 'F7_1', 'F9_1', 'FT7_1', 'FC3_1', 'FC1_1', 'Cz_1',
       'C3_1', 'T7_1', 'CP3_1', 'Pz_1', 'P3_1', 'P7_1', 'PO9_1', 'O1_1',
       'FCZ_1', 'PO10_1', 'P8_1', 'P4_1', 'CP4_1', 'TP10_1', 'T8_1',
       'C4_1', 'FT8_1', 'FC4_1', 'FC2_1', 'F4_1', 'F8_1', 'F10_1', 'V1_1', 'V2_1']#Electrode 17 is FCZ not O2"""

"""electrodes.extend(['FZ_2', 'F3_2', 'F7_2', 'F9_2', 'FT7_2', 'FC3_2', 'FC1_2', 'Cz_2',
       'C3_2', 'T7_2', 'CP3_2', 'Pz_2', 'P3_2', 'P7_2', 'PO9_2', 'O1_2',
       'FCZ_2', 'PO10_2', 'P8_2', 'P4_2', 'CP4_2', 'TP10_2', 'T8_2',
       'C4_2', 'FT8_2', 'FC4_2', 'FC2_2', 'F4_2', 'F8_2', 'F10_2', 'V1_2', 'V2_2'])"""

#exclude = [1,2,28,32,4,6,18,21]#Exclude eye electrodes and malfunctioning electrodes

#exclude.extend([32, 33, 35, 37, 49, 52, 63, 69])
filename = "DualEEG_RPS_C_13.eeg"

In [8]:
vmrk = helper.parse_vmrk("DualEEG_RPS_C_13.vmrk")#To cut off initial part

In [9]:
power1 = helper.frequency_bands_to_common_reference(folder+filename,electrodes,participant =0, vmrk = vmrk, exclude = exclude)
power2 = helper.frequency_bands_to_common_reference(folder+filename,electrodes,participant =1, vmrk = vmrk, exclude = exclude)
power = {**power1,**power2}

............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [13]:
if False:#Check result
    import matplotlib.pyplot as plt
    print("Print result for high correlation electrode F7")
    plt.plot(power["alpha_F7_1"])

In [18]:
with open(folder+"DualEEG_RPS_C_13.power_common_reference","wb") as f:
    pickle.dump(power,f)#1d signal

In [30]:
power1 = {}#Get back power of participant1 only
for k in [k for k in list(power.keys()) if re.search("_1",k)]:
    power1[k] = power[k]

# Compute contrast between all electrodes in alpha, beta and gamma range

In [44]:
import os
import re
import numpy as np
from scipy import ndimage

import sys
import pickle
import json
import tables

import importlib.util #import from superordinate folder
spec = importlib.util.spec_from_file_location("helper", 
                                              str(re.search("(.*/)",os.getcwd()).groups(0)[0])+"helper.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)

In [45]:
folder = "/mnt/c/Users/eler/Documents/Cognitive_Science/Bachelorarbeit/Scripts/processing_pipeline/dyad_13/"
eeg_path = folder+"DualEEG_RPS_C_13.eeg"
vmrk = folder+"DualEEG_RPS_C_13.vmrk"

### Participant 1

In [46]:
data = helper.frequency_bands_between_electrodes(path=eeg_path, participant = 0, vmrk =vmrk)

.........0.001953125
.........0.00390625
.........0.005859375
.........0.0078125
.........0.009765625
.........0.01171875
.........0.013671875
.........0.015625
.........0.017578125
.........0.01953125
.........0.021484375
.........0.0234375
.........0.025390625
.........0.02734375
.........0.029296875
.........0.03125
.........0.033203125
.........0.03515625
.........0.037109375
.........0.0390625
.........0.041015625
.........0.04296875
.........0.044921875
.........0.046875
.........0.048828125
.........0.05078125
.........0.052734375
.........0.0546875
.........0.056640625
.........0.05859375
.........0.060546875
.........0.0625
.........0.064453125
.........0.06640625
.........0.068359375
.........0.0703125
.........0.072265625
.........0.07421875
.........0.076171875
.........0.078125
.........0.080078125
.........0.08203125
.........0.083984375
.........0.0859375
.........0.087890625
.........0.08984375
.........0.091796875
.........0.09375
.........0.095703125
.........0.097656

In [13]:
with open(folder+"RPS_C_13_C3_left.bands","rb") as f:#test loading
    gamma_low_contrasts = np.load(f)

In [6]:
#data = helper.frequency_bands_between_electrodes(path=path, participant = 1, vmrk =vmrk)
with open(folder+"RPS_C_13_C3_left.bands","wb") as f:
    np.save(f, data)
    
df.style.applymap(color_range)

### Participant 2

In [5]:
data = helper.frequency_bands_between_electrodes(path=eeg_path, participant = 1, vmrk =vmrk)
with open(folder+"RPS_C_13_C3_right.bands","wb") as f:
    np.save(f, data)

.........0.001953125
.........0.00390625
.........0.005859375
.........0.0078125
.........0.009765625
.........0.01171875
.........0.013671875
.........0.015625
.........0.017578125
.........0.01953125
.........0.021484375
.........0.0234375
.........0.025390625
.........0.02734375
.........0.029296875
.........0.03125
.........0.033203125
.........0.03515625
.........0.037109375
.........0.0390625
.........0.041015625
.........0.04296875
.........0.044921875
.........0.046875
.........0.048828125
.........0.05078125
.........0.052734375
.........0.0546875
.........0.056640625
.........0.05859375
.........0.060546875
.........0.0625
.........0.064453125
.........0.06640625
.........0.068359375
.........0.0703125
.........0.072265625
.........0.07421875
.........0.076171875
.........0.078125
.........0.080078125
.........0.08203125
.........0.083984375
.........0.0859375
.........0.087890625
.........0.08984375
.........0.091796875
.........0.09375
.........0.095703125
.........0.097656